# Advanced Model Customization and Training

For advanced users who want to customize or train their own models.

**Prerequisites:** PyTorch experience, GPU recommended

**Estimated Runtime:** 20+ minutes (training depends on data size)

In [ ]:
# !pip install promethium-seismic==1.0.3

In [ ]:
import promethium
from promethium import set_seed, get_device, generate_synthetic_traces, add_noise

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

set_seed(42)
device = get_device()
print(f"Promethium {promethium.__version__} | Device: {device}")

## 1. Define Custom Model

In [ ]:
class SimpleDenoiser(nn.Module):
    """Simple convolutional denoiser for demonstration."""
    
    def __init__(self, in_channels=1, hidden=32):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels, hidden, kernel_size=7, padding=3),
            nn.ReLU(),
            nn.Conv1d(hidden, hidden*2, kernel_size=5, padding=2),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Conv1d(hidden*2, hidden, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden, in_channels, kernel_size=7, padding=3),
        )
        
    def forward(self, x):
        return self.decoder(self.encoder(x))

model = SimpleDenoiser().to(device)
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

## 2. Prepare Training Data

In [ ]:
# Generate training data
clean, _ = generate_synthetic_traces(n_traces=500, n_samples=256, seed=42)
noisy = add_noise(clean, noise_level=0.3, seed=42)

# Convert to tensors
X = torch.tensor(noisy, dtype=torch.float32).unsqueeze(1)  # (N, 1, L)
Y = torch.tensor(clean, dtype=torch.float32).unsqueeze(1)

# Split
n_train = 400
train_ds = TensorDataset(X[:n_train], Y[:n_train])
val_ds = TensorDataset(X[n_train:], Y[n_train:])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)

print(f"Train: {len(train_ds)}, Val: {len(val_ds)}")

## 3. Training Loop

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

train_losses, val_losses = [], []
n_epochs = 20

for epoch in range(n_epochs):
    # Train
    model.train()
    train_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_losses.append(train_loss / len(train_loader))
    
    # Validate
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            val_loss += criterion(pred, y).item()
    val_losses.append(val_loss / len(val_loader))
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}: train={train_losses[-1]:.6f}, val={val_losses[-1]:.6f}")

In [ ]:
# Plot learning curves
plt.figure(figsize=(10, 4))
plt.plot(train_losses, label='Train')
plt.plot(val_losses, label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Learning Curves')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4. Save and Load Model

In [ ]:
# Save
torch.save(model.state_dict(), 'custom_denoiser.pt')
print("Model saved")

# Load
loaded_model = SimpleDenoiser().to(device)
loaded_model.load_state_dict(torch.load('custom_denoiser.pt'))
loaded_model.eval()
print("Model loaded")

## Summary

This notebook demonstrated:
1. Custom model architecture
2. Data preparation pipeline
3. Training loop implementation
4. Model checkpointing